# 01: General `lumispy` workflow

## Energy conversion and model fitting

This notebook shows:

- Plotting cathodoluminescence data in different ways (and interactively)
- Converting wavelength signal axis to energy signal axis
(**Required versions: hyperspy>=1.7.0 and lumispy>=0.2**)
- Gaussian fitting

Import packages:

In [ ]:
%matplotlib widget
# Use '%matplotlib widget' in JupterLab and '%matplotlib notebook' in JupyterNotebook for interactive inline functionality (e.g. on binder)
#For pop-up window plots on your local computer, use '%matplotlib tk' or '%matplotlib qt' instead
import lumispy as lum
import hyperspy.api as hs
import os
import matplotlib.pyplot as plt

## Loading the pre-processed CL-SEM file

Load the `.hspy` file, which has already been pre-processed (background subtracted, spike removed and spectral corrected). See microscope-specific tutorials on how to do that.

*You can also leave the path empty. A pop-up window will appear to select the| file from the browser.*

In [ ]:
# Load file
path = os.path.relpath("demo-files/01/01_demo.hspy")
cl_sem = hs.load(path, signal_type='CL_SEM')

print(cl_sem)

In [ ]:
cl_sem.metadata

## Plotting data

Plot the hyperspectral data:

In [ ]:
cl_sem.plot()

Plot the average CL spectrum:

In [ ]:
cl_sem.mean().plot()

### Chromatic imaging:

Plot the panchromatic image:

In [ ]:
cl_sem.T.mean().plot(cmap='viridis')

Get the colour filtered panchromatic images.
Select the energy region to plot as coloured image interactively.

In [ ]:
im = cl_sem.T
im.plot()
roi1 = hs.roi.SpanROI(left=455, right=485) #sets a digitalbandfilter
im_roi1 = roi1.interactive(im, color="red")
im_roi1_mean = hs.interactive(im_roi1.mean,
                          event=roi1.events.changed,
                          recompute_out_event=None)
im_roi1_mean.plot(cmap='viridis')

In [ ]:
im_filtered = roi1(im).mean()

roi_width = roi1.right - roi1.left
roi_centre = roi1.left + 0.5* roi_width

im_filtered.metadata.General.title = "Colour filtered image of {:.0f} $\pm$ {:.0f} nm".format(roi_centre, roi_width)
im_filtered.plot(cmap='viridis')

## Axes types and wavelength to energy conversion

Required versions: hyperspy>=1.7.0 and lumispy>=0.2

HyperSpy has different types of axes:
- The standard `UniformDataAxis` is defined through an `offset` and a `scale` (delta between pixels)
- A `FunctionalDataAxis` is defined through a `UniformDataAxis` and a `function` to convert the values
- A more general `DataAxis` is defined through an `axis` vector/array

The *wavelength* scale of our sample object is a `UniformDataAxis`:

In [ ]:
cl_sem.axes_manager

*LumiSpy* provides easy conversions of the signal axis to the **energy scale**:

It can either replace the axis in the existing object (default) or create a copy of the signal object with the new axis (`inplace=False`):

In [ ]:
cl_sem_eV = cl_sem.to_eV(inplace=False)

The axes manager now contains an `energy` axis, which is a non-uniform `DataAxis`:

In [ ]:
cl_sem_eV.axes_manager

To explore the data in the energy domain, we again plot the signal:

In [ ]:
cl_sem_eV.plot()

### Jacobian transformation

To preserve the integrated intensity per spectral window, a *Jacobian* transformation has to be applied to the signal intensity:

As we require $I(E)dE = I(\lambda)d\lambda$, the scale transformation $E=hc/\lambda$ implies

$$I(E) = I(\lambda)\frac{d\lambda}{dE} = I(\lambda)\frac{d}{dE}
\frac{h c}{E} = - I(\lambda) \frac{h c}{E^2}$$

(where the minus sign just reflects the different directions of integration in the wavelength and energy domains)

This transformation is the default in LumiSpy, but can be deactivated by setting `jacobian=False`.

To visualize the effect of the *Jacobian transformation*, we can plot a signal with constant intensity before and after the transformation:

In [ ]:
# Create a model signal with linear intensity
import numpy as np
axis = {'offset': 300, 'scale': 4, 'units': 'nm', 'size': 101, 'name': 'Wavelength'}
s = hs.signals.Signal1D(np.ones(101), axes=[axis])
s.set_signal_type("Luminescence")
s2 = s.to_eV(inplace=False)

In [ ]:
# Some additional arrays to help with visualizing the spectral bins during plotting
x = np.arange(9)*50+300
x2 = lum.nm2eV(x)
y2 = hs.signals.Signal1D(np.ones(9), axes=[{'offset': 300, 'scale': 50, 'size': 9,}])
y2.set_signal_type("Luminescence")
y2.to_eV()

In [ ]:
# Plot comparative figures
fig1 = plt.figure(figsize=(10,4))
ax0 = plt.subplot(121)
plt.ylim(0,1.3)
plt.xlabel('Wavelength (nm)')
ax0.plot(s.axes_manager[0].axis,s.data,color='orange')
ax0.vlines(x,0,1,color='orange')
ax0.fill_between(s.axes_manager[0].axis,0,s.data, facecolor='orange', alpha=0.3)
ax1 = plt.subplot(122)
plt.ylim(0,0.43)
plt.xlabel('Energy (eV)')
ax1.plot(s2.axes_manager[0].axis,s2.data,color='orange')
ax1.vlines(x2,0,y2.data[::-1],color='orange')
ax1.fill_between(s2.axes_manager[0].axis,0,s2.data, facecolor='orange', alpha=0.3)

## Fitting Gaussian

We will introduce basic fitting functionality, for more details see the `Fitting_tutorial` in the [HyperSpy demos repository](https://github.com/hyperspy/hyperspy-demos).

First, we need to **initialize the model**:

In [ ]:
m = cl_sem_eV.create_model()

**Check the components** of the model (should be empty, but for some types of signals, an automatic background component is added):

In [ ]:
m.components

Now we need to **create some components** and **add them to the model**.

We will use a constant `Offset` and a Gaussian (defined through height and FWHM, hence `GaussianHF`):

In [ ]:
bkg = hs.model.components1D.Offset()
g1 = hs.model.components1D.GaussianHF()
m.extend([g1, bkg])
m.components

To see the parameters of our components and their default values, we can **print all parameter values**:

In [ ]:
m.print_current_values()

Lets set some **sensible starting values** for our components, for a position in the map where we know that there should be signal (as it is not the case everywhere for our sample dataset):

In [ ]:
cl_sem_eV.axes_manager.indices = (7,7)
g1.centre.value = 2.4        # Gaussian centre
g1.fwhm.value = 0.1      # Gaussian width
g1.height.value = 5      # Gaussian height
bkg.offset.value = 0.1   # background offset

We can also **set boundaries** (`bmin` and `bmax`) for some of the parameters:

In [ ]:
g1.centre.bmax = g1.centre.value + 0.2
g1.centre.bmin = g1.centre.value - 0.2
g1.fwhm.bmin = 0.01

We can now **fit the model at the chosen position**, copy the result as starting value to all positions, and **plot** the result:

In [ ]:
m.fit()
m.assign_current_values_to_all()
m.plot()

Again, we can also **print the updated parameters**:

In [ ]:
m.print_current_values()

The model now has the result from our chosen pixel everywhere. Using this as optimized starting paramters, we can now **fit all pixels**. When plotting, we activate additional plotting of the individual components:

In [ ]:
m.multifit(bounded=True, show_progressbar=True)
m.plot(plot_components=True)

To plot maps of the parameters of the Gaussian, we create signal objects from these datasets:

In [ ]:
m_centre = g1.centre.as_signal()
m_centre.plot(cmap='bwr_r', centre_colormap=False) # Otherwise, it would be centred around `0` and we would see little difference between pixels
m_intensity = g1.height.as_signal()
m_intensity.plot(cmap='viridis')

### Particle segmentation

We can use the fit model as basis to do a particle segmentation by **creating a mask** for all pixels, where the intensity is below the mean value:

In [ ]:
mask_treshold = m_intensity.data.mean()
mask = m_intensity.data > mask_treshold #Returns a boolean matrix mask
plt.figure()
plt.imshow(mask)

We can now plot the previous graph of the centre-parametre, after applying the mask:

In [ ]:
(m_centre * mask).plot(cmap='bwr_r', vmin=2.3, centre_colormap=False) 

## END
